In [15]:
import numpy as np
import cv2
import os
from imutils.object_detection import non_max_suppression
from PIL import Image
from statistics import mean
import math  
from playsound import playsound
import pickle

In [16]:
def num_of_people(array):
    if isinstance(array, (np.ndarray, np.generic) ) == True:
        num_p = array.shape[0]
        print("# of People in Frame: {}".format(num_p))
        return num_p
    else:
        pass

In [17]:
def calc_dist_x(array, el_array):
    dist_l = []
    if isinstance(array, (np.ndarray, np.generic) ) == True:
        for i in array:
            h1 = abs(i[1]-i[3])
            h2 = abs(el_array[1]-el_array[3])
            h_ref = max(h1,h2)
            
            dH1 = i[0]-el_array[0]
            dH2 = i[2]-el_array[2]
            dH_av = abs(mean([dH1,dH2]))
            dH_av = dH_av/(2*h_ref)
            
            #print("h_ref: ",h_ref)
            #print("dH_av: ",dH_av)
            if dH_av > 0:
                dist_l.append(dH_av)

        if len(dist_l)>0:
            #print(dist_l)
            return min(dist_l)
        else:
            return 3
    else:
        return 3


In [18]:
# Cargar el pickle
xgbc = []
with (open("xgbc100x100", "rb")) as openfile:
    while True:
        try:
            xgbc.append(pickle.load(openfile))
        except EOFError:
            break
xgbc=xgbc[0]

In [19]:
# initialize the HOG descriptor/person detector
body_det = cv2.HOGDescriptor()
body_det.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

face_det = cv2.CascadeClassifier(os.path.join('haarcascade_frontalface_default.xml'))

cv2.startWindowThread()

# open webcam video stream
cap = cv2.VideoCapture(0)

# the output will be written to output.avi
out = cv2.VideoWriter(
    'output.avi',
    cv2.VideoWriter_fourcc(*'MJPG'),
    15.,
    (640,480))

In [ ]:
label_d = {0:"No_Mask",1:"Mask"}
color_d = {0: (0,0,255), 1: (255,0,0)}

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # resizing for faster detection
    frame = cv2.resize(frame, (640, 480))
    # flipping image to mirror
    frame =cv2.flip(frame,1,1) 
    
    # using a greyscale picture, also for faster detection
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    
    # detect people in the image
    # returns the bounding boxes for the detected people
    # TRY WITH GRAY AND FRAME
    body_boxes, weights = body_det.detectMultiScale(gray, winStride=(8,8) )
    body_boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in body_boxes])
    
    # detect faces in the image
    # returns the bounding boxes for the detected faces
    face_boxes = face_det.detectMultiScale(gray)
    face_boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in face_boxes])
    
    # apply non-maxima suppression to the bounding boxes using overlap threshold to try to maintain overlapping
    # boxes that are still identificable
    body_pick = non_max_suppression(body_boxes, probs=None, overlapThresh=0.5)
    face_pick = non_max_suppression(face_boxes, probs=None, overlapThresh=0.5)
    
    # Testing Results
    #print("body_pick: ",body_pick)
    #print("type: ",type(body_pick))
    #print("dimentions: ", body_pick.shape)
    
    # Checking number of people in frame
    #num_p = num_of_people(face_pick)
    
    
    
    for element in body_pick:
        (xA, yA, xB, yB) = element
        min_dist = calc_dist_x(body_pick, element)
        print("min_dist: ",min_dist)
        if (min_dist > 0) & (min_dist < .2):
            color = 0
        else:
            color = 1
        
        # display the detected boxes in the colour picture
        cv2.rectangle(frame, (xA, yA), (xB, yB), color_d[color], 2)
        cv2.putText(frame, "human", (xA, yA-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
        
        #print("points: {},{},{},{}".format(xA, yA, xB, yB))
        
    for element in face_pick:
        (xA, yA, xB, yB) = element
        
        # Saving FACE IMAGE to a VAR
        face_img = gray[yA:yB, xA:xB]
        resized=cv2.resize(face_img,(100,100))
        #cv2.imwrite("face.jpg", resized)
        #pred = xgbc.predict(resized)
        #print("prediction: ",pred)
        
        # display the detected boxes in the colour picture
        cv2.rectangle(frame, (xA, yA), (xB, yB), (255,255,255), 2)
        cv2.putText(frame, "face", (xA, yA-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
        
        
        
    # Write the output video 
    out.write(frame.astype('uint8'))
    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
# and release the output
out.release()
# finally, close the window
for i in range(1,10):
    cv2.destroyAllWindows()
    cv2.waitKey(1)

min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  0.3793859649122807
min_dist:  0.3793859649122807
min_dist:  0.3760869565217391
min_dist:  0.3760869565217391
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  0.41273584905660377
min_dist:  0.41273584905660377
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  3
min_dist:  0.3606194690265487
min_dist:  0.3606194690265487
min_dist:  3
min_

min_dist:  0.2645739910313901
min_dist:  0.2645739910313901
min_dist:  0.27419354838709675
min_dist:  0.27419354838709675
min_dist:  0.265625
min_dist:  0.265625
min_dist:  0.2800925925925926
min_dist:  0.2800925925925926
min_dist:  0.2599118942731278
min_dist:  0.2599118942731278
min_dist:  0.2800925925925926
min_dist:  0.2800925925925926
min_dist:  0.2694063926940639
min_dist:  0.2694063926940639
min_dist:  0.27293577981651373
min_dist:  0.27293577981651373
min_dist:  0.271889400921659
min_dist:  0.271889400921659
min_dist:  0.271889400921659
min_dist:  0.271889400921659
min_dist:  0.27149321266968324
min_dist:  0.27149321266968324
min_dist:  0.2669683257918552
min_dist:  0.2669683257918552
min_dist:  0.26339285714285715
min_dist:  0.26339285714285715
min_dist:  0.26576576576576577
min_dist:  0.26576576576576577
min_dist:  0.27293577981651373
min_dist:  0.27293577981651373
min_dist:  0.2681818181818182
min_dist:  0.2681818181818182
min_dist:  0.2681818181818182
min_dist:  0.268181818

min_dist:  0.20362903225806453
min_dist:  0.20362903225806453
min_dist:  0.21767241379310345
min_dist:  0.21767241379310345
min_dist:  0.2056451612903226
min_dist:  0.2056451612903226
min_dist:  0.22058823529411764
min_dist:  0.22058823529411764
min_dist:  0.20967741935483872
min_dist:  0.20967741935483872
min_dist:  0.20766129032258066
min_dist:  0.20766129032258066
min_dist:  0.20916334661354583
min_dist:  0.20916334661354583
min_dist:  0.20610687022900764
min_dist:  0.20610687022900764
min_dist:  0.37786259541984735
min_dist:  0.20916334661354583
min_dist:  0.20916334661354583
min_dist:  0.23983739837398374
min_dist:  0.23983739837398374
min_dist:  0.22868217054263565
min_dist:  0.22868217054263565
min_dist:  0.22674418604651161
min_dist:  0.22674418604651161
min_dist:  0.22674418604651161
min_dist:  0.22674418604651161
min_dist:  3
min_dist:  0.2713004484304933
min_dist:  0.2713004484304933
min_dist:  0.2676991150442478
min_dist:  0.2676991150442478
min_dist:  0.2713004484304933
mi

min_dist:  0.2702702702702703
min_dist:  0.2702702702702703
min_dist:  0.2713004484304933
min_dist:  0.2713004484304933
min_dist:  0.2544642857142857
min_dist:  0.2665198237885463
min_dist:  0.2544642857142857
min_dist:  0.24890829694323144
min_dist:  0.2713004484304933
min_dist:  0.24890829694323144
min_dist:  0.2643171806167401
min_dist:  0.2511013215859031
min_dist:  0.2511013215859031
min_dist:  0.25669642857142855
min_dist:  0.2700892857142857
min_dist:  0.25669642857142855
min_dist:  0.2700892857142857
min_dist:  0.25333333333333335
min_dist:  0.25333333333333335
min_dist:  0.2511013215859031
min_dist:  0.2713004484304933
min_dist:  0.2511013215859031
min_dist:  0.25555555555555554
min_dist:  0.2688888888888889
min_dist:  0.25555555555555554
min_dist:  0.25
min_dist:  0.2676991150442478
min_dist:  0.25
min_dist:  0.2688888888888889
min_dist:  0.25333333333333335
min_dist:  0.25333333333333335
min_dist:  0.25
min_dist:  0.2721238938053097
min_dist:  0.25
min_dist:  0.2700892857142

min_dist:  0.2543859649122807
min_dist:  0.18099547511312217
min_dist:  0.18099547511312217
min_dist:  0.18
min_dist:  0.18
min_dist:  0.25555555555555554
min_dist:  0.1872146118721461
min_dist:  0.1872146118721461
min_dist:  0.26146788990825687
min_dist:  0.18385650224215247
min_dist:  0.18385650224215247
min_dist:  0.25675675675675674
min_dist:  0.24892703862660945
min_dist:  0.1832579185520362
min_dist:  0.1832579185520362
min_dist:  0.25
min_dist:  0.18493150684931506
min_dist:  0.18493150684931506
min_dist:  0.18303571428571427
min_dist:  0.18303571428571427
min_dist:  0.2602739726027397
min_dist:  0.25
min_dist:  0.18636363636363637
min_dist:  0.18636363636363637
min_dist:  0.2533632286995516
min_dist:  0.18385650224215247
min_dist:  0.18385650224215247
min_dist:  0.24565217391304348
min_dist:  0.18949771689497716
min_dist:  0.18949771689497716
min_dist:  0.18444444444444444
min_dist:  0.18444444444444444
min_dist:  0.25333333333333335
min_dist:  0.24678111587982832
min_dist:  0.

...

min_dist:  0.19207317073170732
min_dist:  0.3867924528301887
min_dist:  0.19207317073170732
min_dist:  0.18862275449101795
min_dist:  0.37962962962962965
min_dist:  0.18862275449101795
min_dist:  0.1927710843373494
min_dist:  0.38317757009345793
min_dist:  0.1927710843373494
min_dist:  0.18404907975460122
min_dist:  0.3915094339622642
min_dist:  0.18404907975460122
min_dist:  0.19135802469135801
min_dist:  0.38317757009345793
min_dist:  0.19135802469135801
min_dist:  0.19135802469135801
min_dist:  0.3813953488372093
min_dist:  0.19135802469135801
min_dist:  0.1951219512195122
min_dist:  0.3773148148148148
min_dist:  0.1951219512195122
min_dist:  0.18902439024390244
min_dist:  0.37962962962962965
min_dist:  0.18902439024390244
min_dist:  0.19135802469135801
min_dist:  0.3778801843317972
min_dist:  0.19135802469135801
min_dist:  0.19207317073170732
min_dist:  0.38317757009345793
min_dist:  0.19207317073170732
min_dist:  0.1901840490797546
min_dist:  0.3855140186915888
min_dist:  0.190184

min_dist:  0.6863207547169812
min_dist:  0.6863207547169812
min_dist:  0.3691860465116279
min_dist:  0.38425925925925924
min_dist:  0.3691860465116279
min_dist:  0.6613636363636364
min_dist:  0.6613636363636364
min_dist:  0.6976190476190476
min_dist:  0.6976190476190476
min_dist:  0.6813953488372093
min_dist:  0.6813953488372093
min_dist:  0.6689497716894978
min_dist:  0.06621004566210045
min_dist:  0.06621004566210045
min_dist:  0.6720183486238532
min_dist:  0.05504587155963303
min_dist:  0.05504587155963303
min_dist:  0.6910377358490566
min_dist:  0.6910377358490566
min_dist:  0.6863207547169812
min_dist:  0.10377358490566038
min_dist:  0.10377358490566038
min_dist:  0.6822429906542056
min_dist:  0.09345794392523364
min_dist:  0.09345794392523364
min_dist:  0.6947115384615384
min_dist:  0.6947115384615384
min_dist:  0.6590909090909091
min_dist:  0.6590909090909091
min_dist:  0.672093023255814
min_dist:  0.672093023255814
min_dist:  0.6790697674418604
min_dist:  0.6790697674418604
min

min_dist:  0.3860103626943005
min_dist:  0.3860103626943005
min_dist:  0.36650485436893204
min_dist:  0.36650485436893204
min_dist:  0.3963730569948187
min_dist:  0.36650485436893204
min_dist:  0.36650485436893204
min_dist:  0.38082901554404147
min_dist:  0.38082901554404147
min_dist:  0.36650485436893204
min_dist:  0.36650485436893204
min_dist:  0.36650485436893204
min_dist:  0.36650485436893204
min_dist:  0.38082901554404147
min_dist:  0.38082901554404147
min_dist:  0.36650485436893204
min_dist:  0.36650485436893204
min_dist:  0.36650485436893204
min_dist:  0.36650485436893204
min_dist:  0.36650485436893204
min_dist:  0.36650485436893204
min_dist:  3
min_dist:  0.38481675392670156
min_dist:  0.3507853403141361
min_dist:  0.3507853403141361
min_dist:  3
min_dist:  0.3700980392156863
min_dist:  0.3700980392156863
min_dist:  0.36650485436893204
min_dist:  0.36650485436893204
min_dist:  0.0703883495145631
min_dist:  0.32105263157894737
min_dist:  0.0703883495145631
min_dist:  0.366504854

min_dist:  0.22279792746113988
min_dist:  0.10975609756097561
min_dist:  0.10975609756097561
min_dist:  0.22395833333333334
min_dist:  0.22395833333333334
min_dist:  0.24083769633507854
min_dist:  0.35602094240837695
min_dist:  0.08928571428571429
min_dist:  0.08928571428571429
min_dist:  0.24352331606217617
min_dist:  0.24352331606217617
min_dist:  0.24352331606217617
min_dist:  0.08235294117647059
min_dist:  0.38860103626943004
min_dist:  0.08235294117647059
min_dist:  0.2511961722488038
min_dist:  0.04066985645933014
min_dist:  0.04066985645933014
min_dist:  0.25245098039215685
min_dist:  0.25245098039215685
min_dist:  0.26424870466321243
min_dist:  0.26424870466321243
min_dist:  0.25906735751295334
min_dist:  0.25906735751295334
min_dist:  0.2654639175257732
min_dist:  0.2654639175257732
min_dist:  0.25647668393782386
min_dist:  0.25647668393782386
min_dist:  0.2657894736842105
min_dist:  0.2657894736842105
min_dist:  0.25906735751295334
min_dist:  0.25906735751295334
min_dist:  0.

min_dist:  0.18181818181818182
min_dist:  0.18181818181818182
min_dist:  0.18577981651376146
min_dist:  0.18243243243243243
min_dist:  0.18243243243243243
min_dist:  0.18577981651376146
min_dist:  0.18433179723502305
min_dist:  0.18899521531100477
min_dist:  0.18433179723502305
min_dist:  0.18181818181818182
min_dist:  0.19230769230769232
min_dist:  0.18181818181818182
min_dist:  0.182648401826484
min_dist:  0.182648401826484
min_dist:  0.18433179723502305
min_dist:  0.18925233644859812
min_dist:  0.18181818181818182
min_dist:  0.18181818181818182
min_dist:  0.18243243243243243
min_dist:  0.18243243243243243
min_dist:  0.18691588785046728
min_dist:  0.18099547511312217
min_dist:  0.18099547511312217
min_dist:  0.18577981651376146
min_dist:  0.18181818181818182
min_dist:  0.17903930131004367
min_dist:  0.17903930131004367
min_dist:  0.18663594470046083
min_dist:  0.18433179723502305
min_dist:  0.18433179723502305
min_dist:  0.19014084507042253
min_dist:  0.1934673366834171
min_dist:  0.

In [6]:
# When everything done, release the capture
cap.release()
# and release the output
# out.release()
# finally, close the window
for i in range(1,10):
    cv2.destroyAllWindows()
    cv2.waitKey(1)

# Codigo Alberto para mejor lectura

In [ ]:
from PIL import Image
import os
from os import listdir
from os.path import isfile, join
import numpy as np
from tqdm.notebook import tqdm
import cv2
import pickle
from sklearn.model_selection import train_test_split as tts
import pandas as pd
from time import sleep
from IPython.display import clear_output
import numpy as np
xVectorSize=100
yVectorSize=100
def getImageVector(img):
    #img = Image.open(img)
    img = img.resize( (xVectorSize, yVectorSize) )
    img = img.convert('L')
    return imgToVector(img) 
#Funcion para vectorizar imagenes y asignales clasificación
def imgToVector(img,x=xVectorSize,y=yVectorSize,classification=1):
    vector=[]
    vector.append(classification)
    for j in range(y):
        for i in range(x):
            vector.append(img.getpixel((i,j)))
    return vector
def reconstructor(vector,x=xVectorSize,y=yVectorSize):
    reb=Image.new('L',(x,y))
    for j in range(y):
        for i in range(x):
            reb.putpixel((i,j),vector[i+j*x]) 
    return reb
def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)
def faceslocation(imgpath,foi='file'):
    if foi=='file':
        original_image = cv2.imread(imgpath)
    elif foi=="img":
        original_image = imgpath
    
    
    grayscale_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
    detected_faces = face_cascade.detectMultiScale(grayscale_image)
    return detected_faces
def getfaces(imgpath, foi='file'):
    
    if foi == 'file':
        source = Image.open(imgpath)
        source = source.convert('L')
        detected_faces=faceslocation(imgpath)
    elif foi == 'img':
        source = cv2.cvtColor(imgpath, cv2.COLOR_BGR2GRAY)
        detected_faces=faceslocation(imgpath,foi="img")
        
    faces=[]
    
    for i in detected_faces:
        tlbr=list(i)
        
        x=int(tlbr[2])
        y=int(tlbr[3])
        x_offset=int(tlbr[0])
        y_offset=int(tlbr[1])
        
        if foi == 'file':
            face=Image.new('L',(x,y))
            for j in range(y):
                for i in range(x):
                    pixel=source.getpixel((i+x_offset,j+y_offset))
                    face.putpixel((i,j),pixel) 
            faces.append(face)
        elif foi == 'img':
            face=np.array()
            for j in range(y):
                for i in range(x):
                    pixel=source[i+x_offset,j+y_offset]
                    face.putpixel((i,j),pixel) 
            faces.append(face)
    
    mask=[] 
    for face in faces:
        X=pd.DataFrame(getImageVector(face)).T
        X=X.drop(0,axis=1)
        pred=xgbc.predict(X)
        mask.append(pred)
           
    return faces,mask
def textInImage(img,text='Prueba Texto'):
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (50,50)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 2
    cv2.putText(img,text, 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)
    
def idSquare(imgpath,fl,pred,foi='file'):
    if foi == 'file':
        img = cv2.imread(imgpath)
    elif foi == 'img':
        img=imgpath
        
    for i in range(len(fl)):
        
        column=fl[i][0]
        row=fl[i][1]
        width=fl[i][2]
        height=fl[i][3]
        
        if pred[i][0]== 0:
            color=(0,0,255)
        else:
            color =(0,255,0)
        
        cv2.rectangle(
            img,
            (column, row),
            (column + width, row + height),
            color,
            2)
    
    return img
def maskdetector(imgpath,foi='file'):
    if foi == 'file':
        original_image = cv2.imread(imgpath)
        detected_faces=faceslocation(imgpath)
        faces,mask=getfaces(imgpath)
        img=idSquare(imgpath,detected_faces,mask)
    elif foi == 'img':
        detected_faces=faceslocation(imgpath,foi='img')
        faces,mask=getfaces(imgpath,foi='img')
        img=idSquare(imgpath,detected_faces,mask, foi='img')
        
    return img
2:27
El siguiente bloque es el que carga el pickle
2:27
xgbc = []
with (open("xgbc100x100", "rb")) as openfile:
    while True:
        try:
            xgbc.append(pickle.load(openfile))
        except EOFError:
            break
xgbc=xgbc[0]
2:27
y el último es el detector
2:28
cam = cv2.VideoCapture(0)
cv2.startWindowThread()
cv2.namedWindow("test")
stop=False
while stop==False:
    ret, frame = cam.read()
    imgpath='current_img.png'
    cv2.imwrite(imgpath, frame)
    
    #clear_output(wait=True)
    img=maskdetector(imgpath)
    #img=maskdetector(frame,'img')
    try:
        frame50 = rescale_frame(img,percent=50)
        cv2.imshow("test", frame50)
    except:
        pass
        
    
     
    k = cv2.waitKey(1)
    if k%256 == 27: # ESC pressed
        print("Escape hit, closing...")
        stop=True
    
cam.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)
print('End')